In [ ]:
#redesigning code for parallel processing
import matlab.engine
import numpy as np 
import pandas as pd
import math
import matplotlib.pyplot as plt
import os
import time
from sklearn import preprocessing
from scipy import stats
import place_cell_multifunctions as pcm 
from functools import partial
import multiprocessing as mp


#parallel processing works for a function operating on dataset 
#start with mutual information pre-shuffle

events_data=pd.read_csv(r'C:\Users\fgobbo\Meera Thesis\H2203_translated_events_data\H2203_PAL_events_translate.csv', sep=',', error_bad_lines=False, index_col=False, dtype='unicode')

Session='P02'
Stage='PRE'


    
ss_data=events_data[events_data['Session']==Session]
ss_data=ss_data[ss_data['stage']== Stage]
ss_data=ss_data[ss_data['Movement status']=='moving']

cell_events=ss_data[[col for col in ss_data if col.startswith(' C')]]

x_bins=np.linspace(4.8, 94.8, 9, endpoint=False)
x_bins = np.insert(x_bins, 0, -5.2, axis=0)
x_bins = np.insert(x_bins, 10, 94.8, axis=0)
x_bins = np.insert(x_bins, 11, 104.8, axis=0)
y_bins=np.linspace(3.6, 93.6, 9, endpoint=False)
y_bins = np.insert(y_bins, 0, -6.4, axis=0)
y_bins = np.insert(y_bins, 10, 93.6, axis=0)
y_bins = np.insert(y_bins, 11, 103.6, axis=0)
y_bins = np.insert(y_bins, 12, 113.6, axis=0)
y_bins = np.insert(y_bins, 13, 123.6, axis=0)


#x, y coord read in 
x=ss_data['x']
y=ss_data['y']
 #formating events data
cell_events=cell_events.astype(float)

normed_cells=[]
for j in cell_events:
    cellist=cell_events[j].tolist()
    mino=min(cellist)
    maxo=max(cellist)
    sing=[]
    if maxo!=0:
        for i in cellist:
            sing.append((i-mino)/(maxo-mino))
        normed_cells.append(sing)
    else:
        normed_cells.append(cellist)
cell_events=pd.DataFrame(normed_cells)
cell_events=cell_events.transpose()

cell_events=cell_events.astype(float)




#cell_events=cell_events.reset_index(drop=True)
#binarise data
onevals=cell_events>=0.3
zerovals=cell_events<0.3
cell_events[onevals]=1
cell_events[zerovals]=0
#binarise data

#drop any cells with an insufficient number of events
dropped_cells=[]
for i in cell_events:
    if cell_events[i].sum()<3:
        cell_events=cell_events.drop(i, 1)
        dropped_cells.append(i)

new_cell_index=cell_events.columns.tolist()

newx=[]
newy=[]
for i in x:
    newx.append(float(i))
for j in y:
    newy.append(float(j))

x_b=x.tolist()
y_b=y.tolist()
for i in x_b:
    i = float(i)
for i in y_b:
    i = float(i)

occupancy_map_x=[]
occupancy_map_y=[]
for x_c, y_c in zip(newx,newy):
    for val in range(len(y_bins)):
        if val<=len(y_bins)-1:
            if y_bins[val]<=y_c<=y_bins[val+1]:
                for valx in range(len(x_bins)):
                    if x_bins[valx]<=x_c<=x_bins[valx+1]:
                        occupancy_map_x.append(valx)
                        occupancy_map_y.append(val)
OM=list(zip(occupancy_map_x, occupancy_map_y))
event_bins=[]
for j in cell_events:
    cebins=[]
    i=0
    while i<len(cell_events[j]):
        if cell_events[j][i]==0:
            cebins.append(99999)
            i+=1
        else:
            loc=OM[i]
            cebins.append(loc)
            i+=1
    event_bins.append(cebins)
all_locs=[] #list of lists - each sublist represents the cells maximal bins 
p=0
while p<len(event_bins): #going through cell by cell
    uniq=[]  #finding all unique firing locations
    q=0
    while q<len(event_bins[p]):
        if event_bins[p][q] not in uniq:
            if event_bins[p][q]!=99999:  #appending unique event locations
                uniq.append(event_bins[p][q])
                q+=1
            else:
                q+=1
        else:
            q+=1
    freq=[]
    for r in uniq:
        freq.append(event_bins[p].count(r))#counting frequency of each event bin
    
    if not freq:
        all_locs.append([[7,7],[5,5],[3,5,6], [6,7,7]])
        p+=1
    else:
        m=max(freq)
        pos=[i for i, j in enumerate(freq) if j == m]
        cell_max_locs=[]
        z=0
        while z<len(pos):
            cell_max_locs.append(uniq[pos[z]])
            z+=1
        all_locs.append(cell_max_locs)
        p+=1
## finding the number of entries into the maximal bin ##
threshold=5
i=0
todrop=[]
while i<len(all_locs):
    if len(all_locs[i])==1:
        for p, q in all_locs[i]:
            j=0
            entries=[]
            while j<len(x):
                if x_bins[p]<=float(x_b[j])<=x_bins[p+1]:
                    if y_bins[q]<=float(y_b[j])<y_bins[q+1]:
                        if float(x_b[j-1])<x_bins[p] or float(x_b[j-1])>x_bins[p+1] or float(y_b[j-1])<y_bins[q] or float(y_b[j-1])>y_bins[q+1]:
                            entries.append(1)
                            j+=1
                        else:
                            j+=1
                    else:
                        j+=1
                else:
                    j+=1
        entries=entries.count(1)
        if entries<threshold:
            todrop.append(new_cell_index[i])
            i+=1
        else:
            i+=1
    else:
        todrop.append(new_cell_index[i])
        i+=1
for i in todrop:
    cell_events=cell_events.drop(i, 1)
    dropped_cells.append(i)

new_cell_index=cell_events.columns.tolist()
event_arr = cell_events.to_numpy().transpose()
occupie=pcm.spatial_occupancy(x,y)
spatial_en=pcm.spatial_entropy(occupie)
cell_events

In [ ]:
import random
shuffles=1000
shuff_inputs=[]
i=0
while i<len(event_arr):
    new_list=event_arr[i].tolist()
    q=0
    while q<len(range(shuffles)):
        new_list=random.sample(new_list, len(new_list))
        shuff_inputs.append(new_list)
        q+=1
    i+=1


In [ ]:
if __name__ == '__main__':
    start_time = time.time()
    p=mp.Pool(processes=5)
    mutual_infos=p.map(partial(pcm.mutual_information, occupied=occupie, spatial_ent=spatial_en), event_arr)
    print(time.time()-start_time)
    p.close()
    p.join()

In [ ]:
if __name__ == '__main__':
    start_time = time.time()
    p=mp.Pool(processes=5)
    shuffled_dist=p.map(partial(pcm.mutual_information, occupied=occupie, spatial_ent=spatial_en), shuff_inputs)
    print(time.time()-start_time)
    p.close()
    p.join()

In [ ]:
percs=pcm.percentile(shuffled_dist, mutual_infos)

In [ ]:
cell_MI={'cell_id':new_cell_index, 'MI':mutual_infos, 'percentile':percs}
cell_MI=pd.DataFrame(cell_MI)
cell_MI.to_csv('E:\Meera\Mutual Information\H2203_P02_PRE_MI.csv')
dist_scores={'dist_vals':shuffled_dist}
dist_scores=pd.DataFrame(dist_scores)
dist_scores.to_csv('E:\Meera\Mutual Information\H2203_P02_PRE_SD.csv')

In [ ]:
len(new_cell_index)

In [ ]:
len(mutual_infos)

In [ ]:
cell_events